# Write results to `tex` files
---

This notebook contains the workings out for generating our results in the $k$-modes write-up.

We will be examining a selection of well-known categorical datasets and running our algorithm against them to test for their ability in:

- **clustering** (using the objective function and silhouette score)
- computational **speed**

### Each set of results will be found by manipulating some `pandas.DataFrame` and then written to a `tex` file.

In [1]:
from kmodes.kmodes import KModes
from sklearn.metrics import silhouette_score

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sbn
import time

%matplotlib inline
sbn.set_palette('colorblind')

# Read in datasets


In [2]:
soybean = pd.read_csv("../data/soybean.csv", na_values=["?", "dna"], dtype="object")
mushroom = pd.read_csv("../data/mushroom.csv", dtype="object")
zoo = pd.read_csv("../data/zoo.csv", dtype="object")
breast_cancer = pd.read_csv("../data/breast_cancer.csv", dtype="object")
vehicle = pd.read_csv("../data/vehicle.csv", dtype="object")

for dataset in [soybean, mushroom, zoo, breast_cancer, vehicle]:
    dataset.dropna(axis=0, inplace=True)


# Plot results


In [3]:
def get_epoch_plot(dataset, name, class_col="class"):

    data, labels = dataset.drop(class_col, axis=1), dataset[class_col]
    k = labels.nunique()

    fig, ax = plt.subplots(1, figsize=(14, 10), dpi=300)

    for init in [
        "cao",
        "huang",
        "random",
        "matching_best",
        "matching_worst",
        "matching_random",
    ]:

        np.random.seed(0)
        km = KModes(k, init=init, n_init=25)
        km.fit_predict(data)
        costs = km.epoch_costs_

        if "matching" not in init:
            marker = "--"
        else:
            marker = "-"

        ax.plot(range(len(costs)), costs, marker, label=init.replace("_", " ").title())
        ax.set_xlabel("Epoch")
        ax.set_ylabel("Cost")
        ax.legend()
        ax.set_title(f"Epoch costs on the {name} dataset with $k={k}$")
        plt.savefig(f'../img/epoch_plot_{name.replace(" ", "_")}.pdf')


In [4]:
# get_epoch_plot(soybean, 'soybean')


In [5]:
# get_epoch_plot(mushroom, 'mushroom')


In [6]:
# get_epoch_plot(zoo, 'zoo animal')


In [7]:
# get_epoch_plot(breast_cancer, 'breast cancer')


# Define functions to get results tables


In [9]:
mush = mushroom.drop("class", axis=1).values

dissim = np.empty((mush.shape[0], mush.shape[0]))
for i, row in enumerate(mush):
    dissim[i, :] = np.sum(mush != row, axis=1)


[ 0.  7.  9. ... 13. 12. 12.]
[ 7.  0.  5. ... 13. 14. 12.]
[ 9.  5.  0. ... 12. 14. 13.]
[ 3.  9.  8. ... 14. 12. 14.]
[10.  8. 11. ... 14. 12. 13.]
[ 8.  3.  6. ... 13. 13. 13.]
[ 8.  5.  3. ... 13. 14. 13.]
[ 9.  7.  2. ... 13. 13. 14.]
[ 5.  8.  9. ... 15. 11. 14.]
[ 7.  5.  5. ... 13. 14. 13.]
[ 9.  3.  5. ... 14. 13. 13.]
[ 7.  5.  6. ... 13. 13. 13.]
[ 8.  3.  5. ... 13. 14. 13.]
[ 4.  7.  8. ... 15. 11. 14.]
[11. 12. 13. ... 14. 11. 14.]
[ 7.  9. 10. ... 13. 12. 13.]
[12. 10. 11. ... 14. 12. 15.]
[ 2.  7.  8. ... 12. 12. 12.]
[ 4.  8.  7. ... 14. 12. 14.]
[ 1.  6.  8. ... 13. 12. 12.]
[ 7.  3.  4. ... 13. 14. 13.]
[ 5.  7.  8. ... 13. 10. 13.]
[ 8.  5.  4. ... 14. 13. 14.]
[10.  5.  3. ... 14. 13. 14.]
[ 7.  7.  3. ... 13. 14. 13.]
[ 6.  7.  6. ... 12. 12. 14.]
[ 9.  3.  4. ... 13. 13. 13.]
[ 9.  5.  3. ... 14. 13. 13.]
[ 5. 10. 11. ... 11. 11. 12.]
[ 9.  7.  9. ... 15. 12. 15.]
[ 9.  4.  2. ... 13. 14. 13.]
[ 3.  7.  8. ... 15. 12. 14.]
[ 9.  4.  3. ... 13. 13. 13.]
[ 9.  8.  

[10.  7.  9. ... 13. 12. 14.]
[ 9.  4.  4. ... 14. 13. 13.]
[10. 12. 13. ... 13. 11. 14.]
[ 7.  2.  3. ... 13. 14. 12.]
[ 5.  7.  8. ... 15. 12. 14.]
[10.  4.  3. ... 13. 13. 14.]
[ 7. 10. 10. ... 12. 10. 12.]
[ 5.  4.  5. ... 13. 14. 12.]
[ 9.  7.  2. ... 13. 13. 14.]
[ 8.  8.  9. ... 12. 12. 14.]
[ 8.  6.  6. ... 14. 13. 14.]
[ 8.  6.  6. ... 14. 13. 14.]
[ 9.  8.  7. ... 13. 12. 13.]
[ 8. 10. 10. ... 12. 12. 13.]
[ 6. 10. 11. ... 13. 12. 12.]
[ 8.  6.  5. ... 13. 13. 14.]
[ 6.  5.  4. ... 12. 14. 12.]
[ 8.  5.  4. ... 14. 13. 14.]
[ 6.  5.  5. ... 13. 14. 12.]
[ 9.  3.  3. ... 13. 14. 13.]
[ 5.  3.  6. ... 13. 14. 12.]
[12. 13. 15. ... 17. 13. 16.]
[10.  8. 10. ... 15. 12. 16.]
[ 4.  8.  9. ... 15. 12. 14.]
[11. 11. 13. ... 16. 13. 15.]
[ 8.  6.  6. ... 14. 13. 14.]
[ 9.  8.  9. ... 16. 12. 15.]
[ 8.  6.  5. ... 13. 13. 14.]
[ 7.  6.  6. ... 14. 13. 13.]
[11. 10. 11. ... 13. 14. 12.]
[ 8.  6.  3. ... 14. 13. 14.]
[ 8.  3.  3. ... 13. 14. 12.]
[ 9.  8. 10. ... 13. 12. 14.]
[11. 10. 1

[ 7. 10.  9. ... 10. 11. 12.]
[12. 11. 13. ... 16. 13. 15.]
[ 8.  8.  9. ... 15. 13. 14.]
[10. 10.  8. ... 15. 12. 16.]
[ 5.  3.  6. ... 13. 14. 12.]
[10.  8. 10. ... 14. 13. 15.]
[ 2.  8. 10. ... 14. 10. 13.]
[10. 11. 10. ... 17. 12. 16.]
[ 7.  4.  4. ... 13. 14. 12.]
[ 9.  3.  5. ... 14. 13. 13.]
[ 8.  3.  5. ... 13. 14. 13.]
[ 9.  8. 11. ... 16. 12. 15.]
[ 9.  8. 10. ... 15. 12. 15.]
[11. 12. 12. ... 15. 13. 15.]
[ 8.  5.  3. ... 13. 14. 13.]
[ 5.  8.  9. ... 15. 11. 14.]
[ 9.  2.  6. ... 14. 13. 13.]
[10. 11. 13. ... 14. 12. 15.]
[ 3.  6.  7. ... 12. 12. 12.]
[ 9.  6.  4. ... 14. 13. 14.]
[ 6.  3.  4. ... 13. 14. 12.]
[11. 10.  9. ... 15. 12. 17.]
[ 6.  1.  6. ... 13. 14. 12.]
[ 9.  6.  4. ... 14. 13. 14.]
[ 9.  6.  9. ... 14. 12. 13.]
[ 7.  3.  5. ... 13. 14. 12.]
[ 6.  9. 10. ... 12. 10. 11.]
[ 7.  3.  4. ... 12. 14. 12.]
[10.  6.  8. ... 15. 13. 14.]
[ 3.  6.  8. ... 13. 12. 12.]
[ 8.  3.  3. ... 13. 14. 12.]
[ 7.  3.  5. ... 13. 14. 12.]
[ 7.  4.  5. ... 14. 13. 13.]
[10.  5.  

[ 7.  8. 10. ... 14. 12. 13.]
[ 7.  6.  6. ... 14. 13. 13.]
[11. 11.  8. ... 15. 12. 17.]
[11. 11. 11. ... 16. 12. 17.]
[ 7.  5.  3. ... 13. 14. 12.]
[10. 10. 10. ... 15. 12. 16.]
[ 7. 11. 11. ... 13. 11. 13.]
[11.  9. 11. ... 16. 12. 17.]
[ 8.  8.  9. ... 14. 12. 13.]
[ 9.  6.  8. ... 15. 13. 14.]
[10.  7.  8. ... 15. 13. 14.]
[ 3.  5.  8. ... 13. 11. 12.]
[ 7.  4.  3. ... 12. 14. 12.]
[ 4.  8.  8. ... 15. 12. 14.]
[ 7.  4.  8. ... 14. 13. 13.]
[ 7.  5.  7. ... 14. 13. 13.]
[10.  9. 11. ... 15. 12. 16.]
[ 5.  7.  9. ... 14. 10. 13.]
[ 8.  4.  3. ... 12. 14. 13.]
[ 7. 11. 10. ... 12. 12. 13.]
[ 7.  2.  6. ... 13. 14. 12.]
[11. 10. 13. ... 16. 12. 15.]
[ 8.  4.  5. ... 13. 13. 13.]
[ 9.  6.  4. ... 14. 13. 14.]
[ 8.  3.  2. ... 13. 14. 13.]
[ 9. 10.  9. ... 16. 12. 15.]
[ 7.  4.  3. ... 13. 14. 13.]
[ 8.  8. 10. ... 13. 12. 14.]
[ 8. 10. 10. ... 13. 12. 13.]
[13. 11. 12. ... 16. 13. 15.]
[ 6. 11. 10. ... 11. 10. 12.]
[ 8. 10.  9. ... 13. 12. 14.]
[10. 10.  9. ... 14. 12. 16.]
[ 8.  3.  

[11. 11. 11. ... 13. 12. 14.]
[ 8.  1.  4. ... 13. 14. 13.]
[13. 11. 12. ... 16. 12. 15.]
[11. 11. 14. ... 16. 12. 15.]
[ 4.  8.  8. ... 12. 11. 13.]
[12. 12. 13. ... 15. 13. 15.]
[ 2.  7.  9. ... 13. 12. 12.]
[12. 11. 12. ... 15. 13. 16.]
[ 8.  4.  6. ... 14. 13. 13.]
[ 9. 11. 13. ... 14. 11. 13.]
[10.  9.  9. ... 12. 13. 12.]
[ 9.  5.  5. ... 14. 13. 14.]
[13. 11. 11. ... 14. 12. 15.]
[11. 10. 11. ... 13. 12. 13.]
[13. 11. 10. ... 13. 12. 15.]
[10.  9. 10. ... 13. 12. 12.]
[13. 12. 12. ... 15. 12. 16.]
[10. 10. 12. ... 15. 12. 14.]
[12. 11. 12. ... 14. 12. 14.]
[12. 11. 11. ... 15. 13. 15.]
[10. 11. 13. ... 14. 13. 15.]
[12. 10. 11. ... 14. 12. 14.]
[13. 13. 15. ... 17. 13. 16.]
[11. 12. 13. ... 13. 12. 14.]
[13. 13. 13. ... 15. 13. 16.]
[ 9.  9.  9. ... 12. 12. 14.]
[ 5.  6.  8. ... 15. 11. 14.]
[12. 12. 13. ... 14. 13. 15.]
[ 5.  8.  8. ... 14. 11. 14.]
[ 8.  5.  4. ... 14. 13. 14.]
[ 9. 10. 11. ... 13. 12. 13.]
[13. 13. 14. ... 15. 12. 16.]
[13. 12. 11. ... 14. 12. 16.]
[12. 13. 1

...

[13. 14. 14. ... 16.  5. 16.]
[17. 16. 16. ... 15. 13. 14.]
[11. 12. 13. ... 15.  7. 14.]
[14. 11. 11. ... 14. 13. 14.]
[14. 15. 15. ... 16.  4. 16.]
[14. 15. 15. ... 15.  6. 15.]
[13. 14. 14. ... 15.  6. 16.]
[13. 15. 15. ... 14.  5. 15.]
[14. 15. 15. ... 15.  7. 16.]
[13. 12. 13. ... 14. 11. 14.]
[10. 12. 12. ... 13.  4. 13.]
[14. 15. 15. ... 15.  7. 15.]
[12. 13. 13. ... 13.  6. 14.]
[15. 12. 12. ... 15. 13. 15.]
[13. 15. 15. ... 15.  4. 16.]
[11. 13. 13. ... 12.  5. 13.]
[13. 12. 12. ...  3. 14.  4.]
[10.  9. 10. ... 14. 13. 13.]
[13. 15. 15. ... 14.  5. 15.]
[15. 16. 16. ... 16.  3. 16.]
[11. 13. 13. ... 11.  5. 12.]
[13. 10.  9. ... 13. 12. 13.]
[14. 15. 16. ... 17.  6. 16.]
[15. 16. 16. ... 14.  6. 15.]
[13. 14. 14. ... 13.  6. 14.]
[12. 14. 14. ... 14.  4. 14.]
[11. 10. 11. ... 13. 13. 12.]
[12.  9.  8. ... 12. 13. 12.]
[13. 14. 14. ... 15.  3. 15.]
[12. 14. 14. ... 13.  1. 14.]
[10. 10. 10. ... 12. 11. 13.]
[12. 14. 14. ... 13.  6. 13.]
[12. 14. 14. ... 13.  2. 13.]
[13. 15. 1

[12. 14. 15. ... 14.  8. 13.]
[12. 13. 13. ... 13.  5. 14.]
[14. 15. 15. ... 14.  7. 15.]
[14. 15. 16. ... 16.  8. 15.]
[13. 14. 14. ... 16.  5. 16.]
[15. 15. 15. ... 14. 13. 14.]
[11. 12. 12. ... 13.  6. 14.]
[13. 15. 15. ... 15.  5. 15.]
[11. 13. 13. ... 13.  3. 14.]
[14. 15. 15. ... 14.  9. 15.]
[13. 10. 11. ... 13. 12. 13.]
[12. 14. 14. ... 14.  4. 14.]
[13. 12. 11. ... 14. 14. 14.]
[14. 15. 15. ... 15.  4. 15.]
[13. 10. 12. ... 14. 13. 13.]
[13. 13. 13. ...  3. 13.  2.]
[13. 14. 14. ... 15.  7. 15.]
[12. 14. 14. ... 13.  3. 13.]
[15. 12. 13. ... 15. 12. 15.]
[11. 13. 14. ... 12.  5. 11.]
[12. 14. 14. ... 13.  3. 13.]
[13. 10. 11. ... 14. 14. 13.]
[14. 15. 15. ... 16.  6. 16.]
[14. 15. 15. ... 14.  7. 14.]
[14. 15. 15. ... 15.  5. 15.]
[13. 14. 14. ... 16.  5. 16.]
[12. 14. 14. ... 14.  4. 14.]
[14. 11. 12. ... 14. 12. 14.]
[15. 17. 17. ... 16.  5. 16.]
[12. 13. 13. ... 13.  5. 13.]
[12.  9. 11. ... 13. 12. 12.]
[13. 13. 13. ...  3. 13.  3.]
[13. 10. 11. ... 13. 13. 13.]
[12. 13. 1

[12. 13. 13. ... 14.  6. 14.]
[11. 13. 13. ... 13.  4. 13.]
[14. 13. 13. ... 15. 13. 15.]
[17. 16. 16. ... 14. 14. 13.]
[ 9.  9.  9. ... 11. 12. 12.]
[14. 16. 16. ... 15.  5. 15.]
[17. 16. 17. ... 14. 13. 14.]
[11. 13. 13. ... 13.  4. 13.]
[18. 16. 16. ... 14. 16. 14.]
[12.  9. 11. ... 13. 13. 12.]
[13. 15. 15. ... 15.  2. 15.]
[12. 14. 14. ... 13.  5. 13.]
[13. 15. 15. ... 13.  5. 14.]
[13. 13. 12. ...  1. 14.  3.]
[13. 15. 15. ... 14.  5. 14.]
[14. 16. 16. ... 15.  5. 16.]
[12. 14. 14. ... 15.  4. 15.]
[14. 15. 15. ... 14.  4. 15.]
[13. 15. 15. ... 15.  5. 15.]
[13. 14. 15. ... 15.  7. 14.]
[14. 16. 16. ... 15.  3. 15.]
[11. 10. 11. ... 13. 14. 12.]
[11. 10. 12. ... 13. 12. 12.]
[13. 12. 13. ... 15. 13. 14.]
[12. 14. 14. ... 14.  6. 14.]
[14. 15. 15. ... 16.  6. 16.]
[14. 15. 15. ... 16.  3. 16.]
[13. 14. 14. ... 14.  7. 14.]
[13. 15. 15. ... 14.  6. 14.]
[14. 16. 16. ... 15.  3. 15.]
[12. 13. 13. ... 15.  3. 15.]
[11. 13. 13. ... 14.  2. 14.]
[12. 13. 13. ... 15.  4. 15.]
[13. 13. 1

[12. 11. 12. ...  4. 13.  2.]
[12. 14. 14. ... 15.  4. 15.]
[14. 15. 15. ... 15.  8. 15.]
[14. 15. 15. ... 16.  4. 16.]
[12. 11. 10. ... 13. 13. 13.]
[12. 14. 14. ... 15.  4. 15.]
[14. 15. 15. ... 16.  6. 16.]
[12. 11. 11. ... 13. 12. 13.]
[14. 16. 16. ... 15.  5. 15.]
[10. 12. 13. ... 14.  3. 13.]
[14. 15. 15. ... 15.  5. 15.]
[11.  8.  9. ... 12. 13. 11.]
[12. 11. 12. ... 14. 13. 13.]
[13. 15. 15. ... 15.  5. 15.]
[14. 14. 14. ... 13. 14. 13.]
[13. 14. 14. ... 15.  4. 15.]
[13. 14. 14. ... 14.  4. 14.]
[13. 13. 12. ...  4. 13.  2.]
[14. 11. 11. ... 14. 12. 14.]
[12. 13. 13. ... 13.  5. 13.]
[11. 13. 13. ... 12.  4. 12.]
[14. 16. 16. ... 15.  5. 16.]
[13. 15. 15. ... 14.  4. 15.]
[13. 13. 13. ...  2. 14.  3.]
[11. 13. 13. ... 14.  2. 14.]
[11. 10. 10. ... 12. 12. 12.]
[13. 13. 12. ...  2. 14.  2.]
[14. 16. 16. ... 16.  4. 16.]
[12. 14. 14. ... 14.  5. 14.]
[14. 15. 15. ... 15.  6. 15.]
[13. 10. 10. ... 13. 11. 13.]
[15. 16. 16. ... 15.  5. 15.]
[10.  9. 10. ... 12. 13. 11.]
[14. 15. 1

[12. 12. 13. ...  2. 14.  1.]
[13. 13. 12. ...  2. 13.  5.]
[13. 13. 12. ...  2. 14.  4.]
[14. 15. 15. ... 16.  5. 16.]
[14. 11. 11. ... 14. 14. 14.]
[13. 14. 14. ... 15.  4. 15.]
[12. 14. 14. ... 14.  3. 14.]
[12. 14. 14. ... 14.  3. 14.]
[12. 14. 14. ... 14.  4. 14.]
[12. 11. 12. ... 14. 12. 13.]
[13. 13. 13. ...  4. 13.  3.]
[14. 16. 16. ... 16.  4. 16.]
[12. 11. 11. ...  4. 13.  4.]
[14. 11. 11. ... 14. 14. 14.]
[13. 13. 12. ...  4. 13.  4.]
[12. 12. 13. ...  5. 13.  3.]
[13. 14. 14. ... 14.  4. 14.]
[11. 13. 13. ... 13.  5. 13.]
[13. 15. 15. ... 14.  7. 14.]
[14. 16. 16. ... 15.  6. 15.]
[13. 15. 15. ... 14.  1. 14.]
[13. 12. 12. ... 14. 13. 14.]
[14. 15. 15. ... 14.  5. 14.]
[13. 13. 13. ...  3. 12.  3.]
[13. 14. 14. ... 16.  5. 16.]
[13. 13. 11. ...  1. 14.  3.]
[13. 13. 13. ...  1. 14.  3.]
[13. 12. 11. ...  3. 13.  4.]
[14. 15. 15. ... 16.  3. 16.]
[13. 13. 12. ...  3. 12.  4.]
[14. 15. 15. ... 17.  4. 17.]
[13. 13. 12. ...  4. 12.  5.]
[15. 12. 12. ... 15. 13. 15.]
[13. 13. 1

In [20]:
def matching_dissim(X, y):

    return np.sum(X != y, axis=1)


def get_clustering_results(dataset, init, seed):

    np.random.seed(seed)

    data = dataset.drop("class", axis=1).values
    n_clusters = dataset["class"].nunique()

    km = KModes(n_clusters, init=init, n_init=1)
    labels = km.fit_predict(data)

    dissim = np.empty((data.shape[0], data.shape[0]))
    for i, row in enumerate(data):
        dissim[i, :] = matching_dissim(data, row)

    silhouette = silhouette_score(dissim, labels, metric="precomputed")

    result = pd.DataFrame(
        {
            "Initialisation": init.replace("_", " ").title(),
            "Initial cost": km.epoch_costs_[0],
            "Final cost": km.cost_,
            "Silhouette score": silhouette,
            "No. of iterations": km.n_iter_,
        },
        index=[""],
    )
    return result


def format_results(results):

    mean_df = results.groupby("Initialisation").mean().reset_index()
    mean_df[""] = "mean"

    std_df = results.groupby("Initialisation").std().reset_index()
    std_df[""] = "std"

    median_df = results.groupby("Initialisation").median().reset_index()
    median_df[""] = "median"

    return (
        pd.concat([mean_df, std_df, median_df])
        .groupby(["Initialisation", ""])
        .sum()
        .T.round(4)
    )


def get_init_results(data, init, max_seed):

    dfs = []
    for seed in range(max_seed):
        result = get_clustering_results(data, init, seed)
        dfs.append(result)

    return pd.concat(dfs)


def get_results_table(dataset, name, max_seed=25):

    clust_results = []
    for init in [
        "cao",
        "huang",
        "random",
        "matching_best",
        "matching_worst",
        "matching_random",
    ]:
        clust_result = get_init_results(dataset, init, max_seed)
        clust_results.append(clust_result)

    results = format_results(pd.concat(clust_results)).reindex(
        ["No. of iterations", "Initial cost", "Final cost", "Silhouette score"]
    )
    results.T.to_latex(f'../tex/{name.replace(" ", "_")}_results.tex')


# Collect results


In [21]:
# get_results_table(soybean, 'soybean')


In [22]:
# get_results_table(mushroom, 'mushroom')


In [23]:
# get_results_table(zoo, 'zoo animal')


In [24]:
# get_results_table(breast_cancer, 'breast cancer')
